## Import Libraries

In [49]:
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import defaultdict

## Grid World Environment Setting

In [50]:
# parameter of environment setting
HEIGHT = 6
WIDTH = 6
TRANSITION_PROB = 1
POSSIBLE_ACTIONS = [0,1,2,3] # left, right, up, down
ACTIONS =[(-1,0), (1,0), (0,-1), (0,1)] # actions represented by grid points.
REWARDS = []

In [57]:
class Env:
    def __init__(self):
        super(Env,self).__init__()
        self.transition_probability = TRANSITION_PROB
        self.width = WIDTH
        self.height = HEIGHT
        self.reward = [[0] * WIDTH for _ in range(HEIGHT)]
        self.possible_actions = POSSIBLE_ACTIONS
        self.n_actions = len(self.possible_actions)
        self.reward[5][5] = 3 # desirable destination
        self.reward[0][3] = -1  
        self.reward[1][0] = -1 
        self.reward[1][4] = -1 
        self.reward[4][3] = -1
        self.reward[5][0] = -1
        self.all_state = []


        for x in range(WIDTH):
            for y in range(HEIGHT):
                state = [x, y]
                self.all_state.append(state)
    
    def get_reward(self, state, action):
        next_state = self.state_after_action(state, action)
        return self.reward[next_state[0]][next_state[1]]

    def state_after_action(self, state, action_index):
        action = ACTIONS[action_index]
        return self.check_boundary([state[0] + action[0], state[1] + action[1]])


    # static method
    @staticmethod
    # TO PREVENT TO MOVE AGENT FORWARDING OUT OF GRID 
    def check_boundary(state):
        state[0] = (0 if state[0] < 0 else WIDTH - 1 if state[0] > WIDTH else state[0] - 1)
        state[1] = (0 if state[1] < 0 else HEIGHT - 1 if state[1] > HEIGHT else state[1] - 1)
        return state

    def get_transition_prob(self, state, action):
        return self.transition_probability

    def get_all_states(self):
        return self.all_state

    def render(self):
        time.sleep(0.03)
        self.update()


## Q - Learning Declaration

In [2]:
import numpy as np
import random
from environment import Env
from collections import defaultdict


class QLearningAgent:
    def __init__(self, actions):
        self.actions = actions
        self.step_size = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.9
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])

    # <s, a, r, s'> 샘플로부터 큐함수 업데이트
    def learn(self, state, action, reward, next_state):
        state, next_state = str(state), str(next_state)
        q_1 = self.q_table[state][action]
        # 벨만 최적 방정식을 사용한 큐함수의 업데이트
        q_2 = reward + self.discount_factor * max(self.q_table[next_state])
        self.q_table[state][action] += self.step_size * (q_2 - q_1)

    # 큐함수에 의거하여 입실론 탐욕 정책에 따라서 행동을 반환
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 무작위 행동 반환
            action = np.random.choice(self.actions)
        else:
            # 큐함수에 따른 행동 반환
            state = str(state)
            q_list = self.q_table[state]
            action = arg_max(q_list)
        return action


# 큐함수의 값에 따라 최적의 행동을 반환
def arg_max(q_list):
    max_idx_list = np.argwhere(q_list == np.amax(q_list))
    max_idx_list = max_idx_list.flatten().tolist()
    return random.choice(max_idx_list)


if __name__ == "__main__":
    env = Env()
    agent = QLearningAgent(actions=list(range(env.n_actions)))

    for episode in range(10):
        state = env.reset()

        while True:
            # 게임 환경과 상태를 초기화
            env.render()
            # 현재 상태에 대한 행동 선택
            action = agent.get_action(state)
            # 행동을 취한 후 다음 상태, 보상 에피소드의 종료여부를 받아옴
            next_state, reward, done = env.step(action)
            # <s,a,r,s'>로 큐함수를 업데이트
            agent.learn(state, action, reward, next_state)

            print('episode : ' + str(episode) + 'The Sequence of action is : ' + str(action))
            print('q-table : ' + '\n' + str(agent.q_table))

            state = next_state
            
            # 모든 큐함수를 화면에 표시
            env.print_value_all(agent.q_table)

            if done:
                break    

TclError: image "pyimage4" doesn't exist

: 